<a href="https://colab.research.google.com/github/lmoss/onesharp/blob/main/Copy_of_primitive_recursion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
import pandas as pd
import numpy as np

def ones(n):
    w = ['1' for i in range(n)]
    return(unparse(w))    


def program_checker(str):
    m = len(str)
    x1 = str[m - 1] == '#'
    x2 = all((str[i] == '1' or str[i] == '#') for i in range(m))
    x3 = (str.find('######') == -1)
    if (x1 and x2 and x3):
        flag = True
    else:
        flag = False
        print('The input ' + str + ' is not a valid 1# program.')
        print('It is not the concatenation of a sequence of instructions in the language.')
        print('So what you are asking for is undefined.')
    return (flag)

def one_or_sharp_check(letter):
    if (letter=="1" or letter=="#"):
        return(True)
    else:
        return(False)

def word_checker(strg):
    answer = all([one_or_sharp_check(x)==True for x in strg])
    return(answer)

def input_checker(input_seq):
    seq = [word_checker(x) for x in input_seq]
    flag = all([word_checker(x) for x in input_seq])
    if not flag:
        print('The input sequence contains words with characters other than 1 and #.')
        print('So what you are asking for is undefined.')
    return(flag)
                                                                                
class Augmented:
    def __init__(self, string, remainders):
        self.string = string
        self.remainders = remainders

class Snapshot:
    def __init__(self, instr_number, regs, proceed,verbose,program_length, step_number):
        self.instr_number = instr_number
        self.regs = regs
        self.proceed = proceed
        self.verbose = verbose
        self.program_length = program_length
        self.step_number = step_number
        
def preparse(xstr):
    b = xstr.string.find('#1')
    xstr.remainders = xstr.remainders + [xstr.string[:(b + 1)]]
    xstr.string = xstr.string[(b + 1):]
    return (xstr)



def parse(y):
    tempx = Augmented(y, [])
    while tempx.string.find('#1') >= 0:
        tempx = preparse(tempx)
    return (tempx.remainders + [tempx.string])

def unparse(p):
    return (''.join(p))

def instruction_type(instruction):
    if instruction[-2:] == '1#':
        return ('add1')
    if instruction[-3:] == '1##':
        return ('add#')
    if instruction[-4:] == '1###':
        return ('forward')
    if instruction[-5:] == '1####':
        return ('backward')
    if instruction[-6:] == '1#####':
        return ('cases')

def tail(list):
    return (list[1:])

def one_step(p, snapshot): # p is parsed
    i = snapshot.instr_number
    r = snapshot.regs
    instruction = p[-1 + i]
    if snapshot.verbose:
        print('Step ' + str(snapshot.step_number) + ':')
        print('Execute instruction ' + str(i) + ':' + " " +
              instruction_gloss(instruction,i-1) 
              + '.')
        if instruction_type(instruction)=='cases':
            billy= len(instruction) - 5
            if snapshot.regs[billy-1] == "":
                print('The register is empty, so we go ahead 1 instruction.')
            elif snapshot.regs[billy-1][0] == "1":
                print('The first symbol in that register is 1,' +
                      ' so we delete that symbol and go forward 2 instructions.')
            elif snapshot.regs[billy-1][0] == "#":
                print('The first symbol in that register is #,' +
                      ' so we delete that symbol and go forward 3 instructions.')     
    t = instruction_type(instruction)
    if t == 'add1':
        snapshot.instr_number = 1 + snapshot.instr_number
        l = len(instruction)
        reg = len(instruction[:(l - 1)])
        snapshot.regs[reg - 1] = snapshot.regs[reg - 1] + '1'
    if t == 'add#':
        snapshot.instr_number = 1 + snapshot.instr_number
        l = len(instruction)
        reg = len(instruction[:(l - 2)])
        snapshot.regs[reg - 1] = snapshot.regs[reg - 1] + '#'
    if t == 'forward':
        l = len(instruction)
        offset = len(instruction[:(l - 3)])
        snapshot.instr_number = offset + snapshot.instr_number
    if t == 'backward':
        l = len(instruction)
        offset = len(instruction[:(l - 4)])
        snapshot.instr_number = (-offset) + snapshot.instr_number
    if t == 'cases':
        l = len(instruction)
        reg = len(instruction[:(l - 5)])
        if snapshot.regs[reg - 1] == '':
            snapshot.instr_number = 1 + snapshot.instr_number
        elif snapshot.regs[reg - 1][0] == '1':
            snapshot.instr_number = 2 + snapshot.instr_number
            snapshot.regs[reg - 1] = tail(snapshot.regs[reg - 1])
        elif snapshot.regs[reg - 1][0] == '#':
            snapshot.instr_number = 3 + snapshot.instr_number
            snapshot.regs[reg - 1] = tail(snapshot.regs[reg - 1])
    if 0< snapshot.instr_number <= len(p):
        snapshot.proceed = True
        if snapshot.verbose == True:
            print_snapshot(snapshot)
    else:
         snapshot.proceed = False
    return (snapshot)


def number_help(instr):
    if instruction_type(instr) == 'add1':
        return (len(instr) - 1)
    if instruction_type(instr) == 'add#':
        return (len(instr) - 2)
    if instruction_type(instr) == 'cases':
        return (len(instr)-5)
    else:
        return (0)


def max_register(p):
    return (max([number_help(instr) for instr in parse(p)]))


def pad(p, register_inputs):
    n = len(register_inputs)
    m = max_register(p)
    extras = ['' for x in range(m - n)]
    bigger = register_inputs + extras
    return (bigger)

def print_snapshot(snap):
    regdf = pd.DataFrame([[snap.regs[n]] for n in range(len(snap.regs))],columns=["contents"])
    regdf.index = np.arange(1, len(regdf) + 1)
    def make_pretty(styler):
        styler.set_properties(**{'background-color': '#FFFFCC'})
        styler.set_properties(**{'text-align': 'left'})
        #styler.set_caption("at the start")
        #styler.hide(axis='index')
        return styler
    display(regdf.style.pipe(make_pretty))  

def step_by_step(word_prog, register_inputs):
    word_prog = word_prog.replace(" ", "")
    register_inputs = [word.replace(" ", "") for word in register_inputs]
    if program_checker(word_prog) and input_checker(register_inputs):
        print('First, here is the program:')
        parse_explain(word_prog)
        print()
        regs = pad(word_prog, register_inputs)
        prog = parse(word_prog)
        N = len(prog)
        snap = Snapshot(1, regs,True,True,N,1)
        print('The computation starts with the register contents shown below.')
        print('The registers include those those which you entered as part of the input')
        print('and also others mentioned in the input program.')
        print_snapshot(snap)
        print()
        while 0 < snap.instr_number < N + 1:
            snap = one_step(prog, snap)
            snap.step_number = (snap.step_number) + 1
        if snap.instr_number <= 0:
            print(
                'The computation has not halted properly ' +
                'because the control went above instruction 1 of the program.'
                 )
        elif (snap.instr_number == (N + 1)) and all(
                snap.regs[i] == ""
                for i in range(1, len(snap.regs))):
            print(
                'The computation then halts properly because' +
                ' the control is just below the last line of the program,')
            print('and because all registers other than R1 are empty.')
            if snap.regs[0] == "":
                print('The output is the empty word.')
            else:
                print('The output is ' + snap.regs[0] + '.')
        else:
            print('This computation does not halt.')
            if snap.instr_number != N + 1:
                print('This is because the program has ' + str(len(prog)) +
                  ' instructions, and control at the end is not one line ' + 
                   'below the bottom of the program.')
                print()
            else:
                not_blank = [
                    i + 1 for i in range(1, len(snap.regs))
                    if snap.regs[i] != ""
                ]
                print('Here is the list of registers whose contents ' +
                      'are not empty at this point, other than R1:' +
                      str(not_blank) + '.')
                print('The register contents at the end are shown above.')


def onesharp(word_prog, register_inputs):
    word_prog = word_prog.replace(" ", "")
    register_inputs = [word.replace(" ", "") for word in register_inputs]  
    if program_checker(word_prog) and input_checker(register_inputs):
        register_inputs = [word.replace(" ", "") for word in register_inputs]
        regs = pad(word_prog, register_inputs)
        prog = parse(word_prog)
        N = len(prog)
        snap = Snapshot(1, regs,True,False, N, 1)
        while snap.proceed:
            snap = one_step(prog, snap)
            snap.step_number = (snap.step_number)+1
        if (snap.instr_number == (N + 1)) and all(
                snap.regs[i] == "" for i in range(1, len(snap.regs))):
            return ((snap.regs)[0])
        else:            
            print("This is undefined.")
            print("The register contents at the end are shown below.")
            print_snapshot(snap)
    else:
        return('undefined')


def instruction_gloss(instr,line):
    if instruction_type(instr) == 'add1':
        return ('add 1 to R' + str(len(instr) - 1))
    if instruction_type(instr) == 'add#':
        return ('add # to R' + str(len(instr) - 2))
    if instruction_type(instr) == 'forward':
        w = len(instr) - 3
        return ('go forward ' + str(w) + ' to instruction ' + str(w+line+1))
    if instruction_type(instr) == 'backward':
        w = len(instr) - 4
        return ('go backward ' + str(w) + ' to instruction ' + str(line - w+1))
    if instruction_type(instr) == 'cases':
        return ('cases on R' + str(len(instr) - 5))

def expanded(gorp):
    pgorp = parse(gorp)
    wwgorp = [[pgorp[x],instruction_gloss(pgorp[x],x)] for x in range(len(pgorp))]
    return(wwgorp)

def parse_explain(prog):
    df = pd.DataFrame(expanded(prog),
                      columns=["instruction", 'explanation'])
    df.index = np.arange(1, len(df) + 1)
    def make_pretty(styler):
                styler.set_properties(**{'background-color': '#C9DFEC'})        
                styler.set_properties(**{'text-align': 'left'})
                return styler
    display(df.style.pipe(make_pretty))
    #display(df)
    
clear_1 = '1#####111###11####111####'

move_2_1 = '11#####111111###111###1##1111####1#111111####'

copy_1_2_3 = '1#####11111111###1111###11##111##11111####11#111#11111111####111#####111111###111###1##1111####1#111111####'

length = '1#####1111111###11####11#1#####111###111111####111####11#####111111###111###1##1111####1#111111####'

write = '1#####111111111###11111###11#11##11##111111####11#11##111111111####11#####111111###111###1##1111####1#111111####'

diag = '1#####11111111111###111111###11##111#111##111##1111111####11#111#111##1111####111#####111111###111###1##1111####1#11####11#####111111###111###1##1111####1#11####'

self = '1#1##1##1##1##1##1#1#1#1#1#1#1#1#1#1#1#1##1##1##1#1#1#1#1#1#1##1##1##1#1#1##1##1#1#1#1##1#1#1#1##1##1#1#1#1##1##1#1#1#1#1#1#1#1##1##1##1##1#1#1##1#1#1#1##1#1#1#1##1##1#1#1#1#1##1##1##1##1#1#1#1##1##1##1##1##1#1#1#1#1#1#1##1##1##1#1#1#1##1##1##1#1##1##1#1#1#1#1##1##1##1##1#1##1#1#1##1##1##1##1#1#1##1##1##1##1##1#1#1#1#1#1#1##1##1##1#1#1#1##1##1##1#1##1##1#1#1#1#1##1##1##1##1#1##1#1#1##1##1##1##1#####11111111111###111111###11##111#111##111##1111111####11#111#111##1111####111#####111111###111###1##1111####1#11####11#####111111###111###1##1111####1#11####'

multiply = '111##1111##11#####11111111###1111###11111##111111##11111####11111#111111#11111111####111111#####111111###111###11##1111####11#111111####111#####11111111###1111###111111##1111111##11111####111111#1111111#11111111####1111111#####111111###111###111##1111####111#111111####11111#####111111###111111111###111111#####11111111111###1111111111###111111####111111#####111111111111111###111111###11111###111111#####111###1111111111111####1###11111#####111###11####111####111111#####1111###11####111####11111#11111#####111###1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111###1#1#####11111111###1111###11111##111111##11111####11111#111111#11111111####111111#####111111###111###1##1111####1#111111####1111#####111111###111###111111##1111####111111#111111####11111#####111###111111###111111111###111111#####11111111111111111111111111111111111###11111111111111111111111111###111111111111111111111111111###111111#####11111111111111111111111###1111111111111111111111111111###111111111111111111111###111111#####111111111111111111111###111111111111111111###1111111111111111111###11111#####111###111111###111111111###111111#####11111111111###1111111111111111###111111111###111111#####1111111111111###1111111111###11111111111###111111#####111###11111111###1###1111111#111111111111111111111111111111111####1111111##11111111111111111111111111111111111####1111111#111111111111111111111####1111111##11111111111111111111111####1###1111111#####111111###111###11111##1111####11111#111111####11111#####111111###111###1111##1111####1111#111111####111#####1111111111111###1111111111###11111#111#####111111###111###11111##1111####11111#111111####1111###11111##1111111111111####11111#11111#####111111###111###111##1111####111#111111####1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111####1#####111###11####111####11#####111###11####111####111#####111###11####111####1111#####111111###111###1##1111####1#111111####'

universal = '1#####1###11###11####111111#1#####111###111111###1111111###11111#####1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111###1###1####111#111111111####111##1#####1111###11###11111###111#11111#####1111111111111111111111111111111111111111111111###111111111###111##1#####1111###11###11111###111#11111#####1111111111111111111111111111111111111###111111111###111##1#####1111###11###11111###111#11111#####1111111111111111111111111111111111111111111111###111111111###111##1#####1111###11###11111###111#11111#####111111111111111111111111111111111111111111111111111111###111111111###111##1#####1111###11###11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111###111#11111#####1111111111###1###111#####111111###111###1111##1111####1111#111111####11111111111111111111111111111111111111111111111111111111111111####111#####1###11###1111###11111#1111#111111####11111##1111##111#####1111111###1111###1111##11111##11111####1111#1111111####111111111111111111111111111111111111111111111111111111111111111111111111111###111#####1###11###11111###11111#1111#111111#1111111####1111##111#####111111###111###1111##1111####1111#111111####11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111####111#####1###11###1111###1111#11111#111111####11111#11111#1111##111#####111111###111###1111##1111####1111#111111####11111#####11111111###11###1###111111#####111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111###1###1###11111111####111111#####1111###111#11111#1111####111#####111###111111#111####1#####111111###111###111##1111####111#111111####1111#####111111###111###1##1111####1#111111####111#####111111###111###1##1111####1#111111####111111111111111111111111111111111111111111111111111111111####11#####111###11111###111111111111###11##11##111111####111#11#####1###111###111##11###111#11111111111111####111##11#####11111111111111111111111111111111111111111111111111111111111111111111###1###111##11111#####1###1111111111111111111111####11111#####1111111111111111111111111111###111111111111111111111111111###11111#####111###11###111111111111111111111111111111111111111111111###11#####111111111111111111###111111111###111#11#####1###1###111#111##111##111111111111111111111111111111111111111111111111111111111###111#11#####1###111###111##11###111#111111111111111111####111#111#1111111111111111111111111111111111111111111111###11#####111111111111111111###111111111###111#11#####1###1###111##111##111##11111111111111111111111111111111111###111#11#####1###111###111##11###111#111111111111111111####111#111##111111111111111111111111###11111#####1###1###11111#####1###1###11#####11111111111111111111111###111###111##1111111111111###11#####1###11111###1###11111#111111#111111###11111#11111#111111#111111#1###11#####111111###111###111##1111####111#111111####111#####111111###111###11##1111####11#111111####1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111####11#####1###1###11#####1###1###11#####11111111111111###11###11111111###11#####1###111###1#11111111####1##1111111111####11#####111###11####111####111111#####11###11####1111#####111###11####111####'

clear_2 = '11#####111###11####111####'
clear_3 = '111#####111###11####111####'
clear_4 = '1111#####111###11####111####'
move_1_2= '1#####111111###111###11##1111####11#111111####'
move_1_3= '1#####111111###111###111##1111####111#111111####'
move_1_4= '1#####111111###111###1111##1111####1111#111111####'
move_2_1= '11#####111111###111###1##1111####1#111111####'
move_2_3= '11#####111111###111###111##1111####111#111111####'
move_2_4= '11#####111111###111###1111##1111####1111#111111####'
move_3_1= '111#####111111###111###1##1111####1#111111####'
move_3_2= '111#####111111###111###11##1111####11#111111####'
move_3_4= '111#####111111###111###1111##1111####1111#111111####'
move_4_1= '1111#####111111###111###1##1111####1#111111####'
move_4_2= '1111#####111111###111###11##1111####11#111111####'
move_4_3= '1111#####111111###111###111##1111####111#111111####'
copy_1_2_3 = '1#####11111111###1111###11##111##11111####11#111#11111111####111#####111111###111###1##1111####1#111111####'
copy_1_2_4 ='1#####11111111###1111###11##1111##11111####11#1111#11111111####1111#####111111###111###1##1111####1#111111####'
copy_1_3_4='1#####11111111###1111###111##1111##11111####111#1111#11111111####1111#####111111###111###1##1111####1#111111####'
copy_2_3_4 = '11#####11111111###1111###111##1111##11111####111#1111#11111111####1111#####111111###111###11##1111####11#111111####'

In [ ]:
#@title
def findme(pair,pair_list): 
  x = [i for i in range(len(pair_list)) if pair_list[i][0]==pair[1]]
  return x[0]

def add_placeholder(line):
  keywords = ['cases','add1','add#','goto','non_label']
  if line[0] in keywords:
    return(['non_label']+line)
  elif line[0][0] in ['1','#']:
    return(['non_label']+line)  
  else:
    return(line)

def flatten(tuple):
  b = len(tuple)
  if b > 1 and tuple[1] == "cases":
    n = tuple[2]
    x = ones(n)+'#####'
    return [[tuple[0],x], [tuple[0]+'@1',tuple[3]], [tuple[0]+'@2',tuple[4]], [tuple[0]+'@3',tuple[5]]]
  elif b > 1 and tuple[1]=='add1':
    return([[tuple[0],ones(tuple[2])+'#']])
  elif b > 1 and tuple[1]=='add#':
    return([[tuple[0],ones(tuple[2])+'##']])
  elif b > 1 and tuple[1]=='goto':
    return [[tuple[0],tuple[2]]]
  elif tuple[1][0] in ['1', '#']:
    k = parse(tuple[1])
    m = len(k)
    return([[tuple[0],k[j]] for j in range(m)])


def ones(n):
  return(unparse(['1' for i in range(n)])) 

def resolve(index,pair_list):
  pair_in_list = pair_list[index]
  #print("pair_in_list " + str(pair_in_list))
  first_char = pair_in_list[1][0]
  #print(first_char)
  if first_char == '1' or first_char == '#':
    return pair_in_list[1]
  elif pair_in_list[1]=='end':
    n = len(pair_list) - index
    return(ones(n)+'###')
  else:
    k = findme(pair_in_list,pair_list)
    if k > index:
      return(ones(k-index)+'###')
    if k < index:
      return(ones(index-k)+'####')


def sanity(line_list):
  w = [add_placeholder(line) for line in line_list]
  #print("[add_placeholder(line) for line in line_list] is")
  #print(w)
  s1 = [flatten(line) for line in w]
  #print('[flatten(line) for line in w] is')
  #print(s1)
  t1 = [item for sublist in s1 for item in sublist]
  #print('[item for sublist in s1 for item in sublist] is')
  #print(t1)
  n = len(t1)
  #print('len of t1 = ' + str(n))
  u1 = [resolve(i,t1) for i in range(n)] 
  return(unparse(u1))      

In [ ]:
#@title
class Tree(object):
    "Generic tree node."
    def __init__(self, name='root', children=None, other='other'):
        self.name = name
        self.other = other
        self.children = []
        if children is not None:
            for child in children:
                self.add_child(child)
    def __repr__(self):
        return self.name
    def add_child(self, node):
        assert isinstance(node, Tree)
        self.children.append(node)
#    *
#   /|\
#  1 2 +
#     / \
#    3   4
t = Tree('*', [Tree('1',[],1),
               Tree('2'),
               Tree('*', [Tree('3'),
                          Tree('4')])],2)

def arity(tree):
  if tree.name == 'z':
    return(1)
  if tree.name == 's':
    return(1)
  if tree.name == 'proj':
    return(tree.other[1])
  if tree.name == 'comp':
    templist = [arity(u) for u in tree.children[1:]]  
    if all_equal(templist):
      return (templist[0])
    else:
      return(999)
  if tree.name == 'plus':
      return(2)    

def maxie(l):
  if l == []:
    return(0)
  else:
    return(max(l))
def height(tree):
  n = maxie([height(u) +1 for u in tree.children])
  return(n)
def all_equal(list):
  if (len(list)==0 or len(list)==1):
    return(True)
  else:
    b = all_equal(list[1:])  
    if b == False:
      return(False)
    else:
      return(list[0]==list[1])

def bump_instr(inst, amount):
  if instruction_type(inst) in ['forward','backward']:  
    return(inst) 
  elif instruction_type(inst) == 'add1':  
    n = number_help(inst)
    return(ones(n+amount)+'#')
  elif instruction_type(inst) == 'add#':  
    n = number_help(inst)
    return(ones(n+amount)+'##')
  elif instruction_type(inst) == 'cases':  
    n = number_help(inst)
    return(ones(n+amount)+'#####')
def bump(prog,amount):
  par = parse(prog)
  t = [bump_instr(instr,amount) for instr in par]
  return(unparse(t))

def clear_prog(n):
   a = ones(n) + '#####'
   b = '111###'
   c = '1###'
   d = '111####'
   return(a+b+c+d)

def move_prog(n,m):
   a = ones(n) + '#####'
   b = '111111###'
   c = '111###'
   d = ones(m)+'##'
   e = '1111####'
   f = ones(m)+'#'
   g = '111111####'
   return(a+b+c+d+e+f+g)

def copy_prog(n,m,p):
   a = ones(n) + '#####'
   b = '11111111###'
   c = '1111###'
   d = ones(m)+'##'
   d1 = ones(p) + '##'
   e = '11111####'
   f = ones(m)+'#'
   f1 = ones(p) + '#'
   g = '11111111####'
   return(a+b+c+d+d1+e+f+f1+g+move_prog(p,n))  

def proj_prog_official(ind,upper):
  index = [k+1 for k in range(ind-1)]    
  #print(index)
  first_part = [clear_prog(j) for j in index]
  #print(first_part)
  if ind == 1:
    middle = []
  else: 
    middle=[move_prog(ind,1)]
  second_index =  [k+ind+1 for k in range(upper-ind)] 
  #print(second_index)
  last_part = [clear_prog(j) for j in second_index]   
  together = first_part + middle + last_part
  done = unparse(together)
  if ind == 1 and upper==1:
    return('1###')
  else:
    return(done)   

def proj_prog(ind,upper):
  return(copy_prog(ind,upper+1, upper+2))

def copy_all_forward(n,m):
  # copies 1 to m, 2 to m+1, . . . n to m+n-1
  a = [copy_prog(i,m+i-1, m+i) for i in range(1,n+1)]
  return(unparse(a))

# bad version below, not 'in place'  
#def compose(f,proglist,n):
  # n is number of registers needed by the fns in the proglist
  #k = len(proglist)
  #a = [copy_all_forward(n,n+i)+bump(proglist[i-1],n+i-1) for i in range(1,k+1)]
  #b = unparse(a)
  #c = b + bump(f,n)
  ## should have a move of register n+1 back to 1 and a clear
  #return(c)



In [ ]:
#@title
u = Tree('comp', [Tree('proj',[],[2,4]),
               Tree('z'),
               Tree('comp', [Tree('s'),
                          Tree('proj',[],[1,4])])],2)
h = Tree('comp', [Tree('z'),Tree('proj',[], [1,2])])
h2 = Tree('proj',[], [2,4])
print(arity(h2))

h1 = Tree('z')
arity(h2)
all_equal([arity(k) for k in h.children])

h5 = Tree('comp',[Tree('add'),Tree('proj',[],[2,4]), Tree('proj',[],[3,4])])
print('h5 arity' + str(arity(h5)))
h6 = Tree('add')
p_2_4 = Tree('proj',[],[2,4])
print(arity(p_2_4))
[5,6,7][1:]
h5.children

4
h5 arity4
4


[add, proj, proj]

In [ ]:
#pr is an index of predecessor in bb.
# One hitch is that pr(##1) will be 11#, not 11 as we would want.
# But pr(1###) would be #, not ####, because pr has a module at the end that 
# rectifies #### to #.
pr = '1#####111###111###111111111111111111111111111111111111111111###11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111###1#####111###1111###11111111111111111111###1##11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111###11#11#1#####111111###111###11##1111####11#111111####11#####111111###111###1##1111####1#111111####1111111111111111111111111111111111111111111111111111111111###11#11##1#####111111###111###11##1111####11#111111####11#####111111###111###1##1111####1#111111####11111111111111111111111111111111111111111###1#####111###1111###11111111111111111111###1##1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111###11##11#1#####111111###111###11##1111####11#111111####11#####111111###111###1##1111####1#111111####111111111111111111###11##11##1#####111111###111###11##1111####11#111111####11#####111111###111###1##1111####1#111111####1###1#####111###1111111111###1111111111111111111111111###11#####111111###111###1##1111####1#111111####111111111111111111111111111111111111111111111111###11##1#####111111###111###11##1111####11#111111####11#####111111###111###1##1111####1#111111####11111111111111111111111111111111###11#1#####111###1111111111###1111111111111111111111111###11#####111111###111###1##1111####1#111111####1111111111111111111###11##1#####111111###111###11##1111####11#111111####11#####111111###111###1##1111####1#111111####111###11#11111111111111111111111111111####1#####111###11111111###11111111111111111111111###1##11#####111###11####111####1111111111111111111###11#1#####111111###111###11##1111####11#111111####11#####111111###111###1##1111####1#111111####111###11##111111111111111111111111111####'

In [ ]:
onesharp(pr,['#11'])

'1#1'

In [ ]:
# s is an index of the successor function in bb
s = '1#####111###111###1111111111111111111111111###1111111111111111111111111111111111111111###11##1#####111###111111111111111111###1111111111111111111###11#1#####111111###111###11##1111####11#111111####11#####111111###111###1##1111####1#111111####1111111111111111111###11##111111111111111111111####11#1#####111111###111###11##1111####11#111111####11#####111111###111###1##1111####1#111111####1###'
h = '#'
for i in range(11):
  h = onesharp(s,[h])
  print(h)

1
#1
11
##1
1#1
#11
111
###1
1##1
#1#1
11#1


In [ ]:
def compare_prog(n,m): # _checks if Rn and Rm have the same thing, 
  #preserving them.   At the end, R(n+1) either has 
  # nothing (if original Rn and Rm are the same) or 1 (if they weren't)
   p = (ones(n) + '#####' + 
     '111###' + 
     '111111###' + 
     '111111111###' + 
     ones(m) + '#####' + 
     '11111111111###' + 
     '11111111111###' + 
     '111111111111###' + 
     ones(m) + '#####' + 
     '111111111111###' + 
     '1111111111111###' + 
     '111111111111111###' + 
     ones(m) + '#####' + 
     '1111111111111111###' + 
     '11111111111111111###' + 
     '1111111111111111111###' + 
     '111111111111111111111111111111111111111111111111111###' + 
     ones(n+2) + '#' + 
     '1111111111111111111###' + 
     ones(n+2) + '##' + 
     '11111111111111111###' + 
     ones(n+1) + '#' + 
     '111111111111111###' + 
     ones(n+1) + '#' + 
     ones(n+2) + '#' + 
     '1111111111111111111111111####' + 
     ones(n+1) + '#' + 
     ones(n+2) + '##' + 
     '111111111###' + 
     ones(n+1) + '##' + 
     '1111111###' + 
     ones(n+1) + '##' + 
     ones(n+2) + '#' + 
     '1111###' + 
     ones(n+1) + '##' + 
     ones(n+2) + '##' + 
     '111111111111111111111111111111111111####' + 
     ones(n) + '#####' + 
     '111111###' + 
     '111###' + 
     ones(n+1) + '##' + 
     '1111####' + 
     ones(n+1) + '#' + 
     '111111####' + 
     ones(n+1) + '#####' + 
     '111111###' + 
     '111###' + 
     ones(n) + '##' + 
     '1111####' + 
     ones(n) + '#' + 
     '111111####' + 
     ones(m) + '#####' + 
     '111111###' + 
     '111###' + 
     ones(n+2) + '##' + 
     '1111####' + 
     ones(n+2) + '#' + 
     '111111####' + 
     ones(n+2) + '#####' + 
     '111111###' + 
     '111###' + 
     ones(m) + '##' +   
     '1111####' + 
     ones(m) + '#' +   
     '111111####' + 
     ones(n+1) + '#' + 
     '111111111111111###' + 
     ones(n+1) + '#####' + 
     '111111###' + 
     '111###' + 
     ones(n) + '##' + 
     '1111####' + 
     ones(n) + '#' + 
     '111111####' + 
     ones(n+2) + '#####' + 
     '111111###' + 
     '111###' + 
     ones(m) + '##' +  
     '1111####' + 
     ones(m) + '#' +   
     '111111####')
   return(p)
 
  

In [ ]:
compare_prog(4,3)
#onesharp(compare_prog(4,3),['111#', '#', '111#', '111#'])

'1111#####111###111111###111111111###111#####11111111111###11111111111###111111111111###111#####111111111111###1111111111111###111111111111111###111#####1111111111111111###11111111111111111###1111111111111111111###111111111111111111111111111111111111111111111111111###111111#1111111111111111111###111111##11111111111111111###11111#111111111111111###11111#111111#1111111111111111111111111####11111#111111##111111111###11111##1111111###11111##111111#1111###11111##111111##111111111111111111111111111111111111####1111#####111111###111###11111##1111####11111#111111####11111#####111111###111###1111##1111####1111#111111####111#####111111###111###111111##1111####111111#111111####111111#####111111###111###111##1111####111#111111####11111#111111111111111###11111#####111111###111###1111##1111####1111#111111####111111#####111111###111###111##1111####111#111111####'

In [ ]:
def successor_prog(to_increment,to_use):
  # gives a program to increment the register 'to_increment' by 1, using 
  # the register 'to_use'
   p = (ones(to_increment) + 
     '#####' + 
     '111###' +
     '111###' +
     '1111111111111111111111111###'+
     '1111111111111111111111111111111111111111###' +
     ones(to_use) + '##' + 
     ones(to_increment) + '#####' + 
     '111###' +
     '111111111111111111###' +
     '1111111111111111111###' +
     ones(to_use) + '#' + 
     ones(to_increment) + '#####' + 
     '111111###' +
     '111###' +
     ones(to_use) + '##' + 
     '1111####' +
     ones(to_use) + '#' + 
     '111111####' +
     ones(to_use) + '#####' + 
     '111111###' +
     '111###' +
     ones(to_increment) + '##' + 
     '1111####' +
     ones(to_increment) + '#' +  
     '111111####' +
     '1111111111111111111###'  +
     ones(to_use) + '##' +  
     '111111111111111111111####' +
     ones(to_use) + '#' +  
     ones(to_increment) + '#####' +  
     '111111###' +
     '111###' +
     ones(to_use) + '##' +  
     '1111####' +
     ones(to_use) + '#' + 
     '111111####' +
     ones(to_use) + '#####' +  
     '111111###' +
     '111###' +
     ones(to_increment) + '##' +  
     '1111####' +
     ones(to_increment) + '#' + 
     '111111####' +
     '1###'
     )
   return(p)    

In [ ]:
len(parse(successor_prog(4,5)))


44

In [ ]:
def pr(g,h,k,free): 
  #n is number of registers = number of arguments to <g,h>
  #free is large enough so that neither g nor h use it or any larger register.
  # g and h are assumed to preserve their inputs
  length_of_move = 7
  length_of_clear = 4
  length_of_compare = 81
  length_of_s = 44
  length_of_h = len(parse(h))
  big_num = (length_of_h + length_of_s + length_of_clear + length_of_move +
             length_of_compare + 2)
  p = (move_prog(k,free) +
      ones(k+1) + '##' +
      h +
      compare_prog(free,k+1) +
      ones(k+2) + '#####' +
      ones(big_num()) + '###' +
      h +
      successor_prog(k+1,k+3) +
      clear_prog(k+2) +
      move_prog(k+2,k+1) +
      compare_prog(free,k+1) +
      ones(big_num) + '####' +
      clear_prog(free)
      )
  return(p)  
  

In [ ]:
w =parse(multiply)
len(w)

182

In [ ]:
first_prog = clear_1 + '11#'
second_prog_prelim = compose(s,[proj_prog(1,2)],4) 
third_prog_prelim = proj_prog(1,2)
second_prog = compose(multiply,[second_prog_prelim,third_prog_prelim], 10)

TypeError: ignored

In [ ]:
#parse_explain(bump(s,2))

onesharp(bump(s,2),['1', '1111#', '11'])
#onesharp(second_prog_prelim,['11','#1'])

This is undefined.
The register contents at the end are shown below.


,contents
1,1
2,1111#
3,##1
4,


In [ ]:
bump(s,2)

'111#####111###111###1111111111111111111111111###1111111111111111111111111111111111111111###1111##111#####111###111111111111111111###1111111111111111111###1111#111#####111111###111###1111##1111####1111#111111####1111#####111111###111###111##1111####111#111111####1111111111111111111###1111##111111111111111111111####1111#111#####111111###111###1111##1111####1111#111111####1111#####111111###111###111##1111####111#111111####1###'

In [ ]:
#a = onesharp(proj_prog(1,2),['11','#1'])
#print(a)
onesharp(bump(s,2),['11','1','11'])

This is undefined.
The register contents at the end are shown below.


,contents
1,11
2,1
3,##1
4,


In [ ]:
def compose(outer_fn,proglist,argument_number,free_reg):
  ## note that the outer_fn has to be 'full', not 'in place'!
  k = len(proglist) # number of functions
  arg = argument_number # = common arity in the proglist
  a = [proglist[i] + move_prog(arg+1,free_reg+i+1) for i in range(0,k)]
  b = unparse(a)
  c = [move_prog(free_reg+i+1,k+i+2) for i in range(0,k)]
  d = unparse(c)
  f = bump(outer_fn,argument_number)
  e = b + d + f   
  return(e)

In [ ]:
proglist = [proj_prog(2,3),proj_prog(1,3)]
clear_reg = 4
argument_number = 3
arg = argument_number
free_reg = 6
k = len(proglist) # number of functions
print(k)
#n = number of arguments that the proglist functions take
a = [proglist[i] + move_prog(arg+1,free_reg+i+1) for i in range(0,k)]
b = unparse(a)
c = [move_prog(free_reg+i+1,k+i+2) for i in range(0,k)]
d = unparse(c)
outer_fn = multiply
f = bump(outer_fn,argument_number)
print(f)
e = b + d + f
#onesharp(f,['11', '#1#','11#','#1#','11#'])
onesharp(e,['11','#1','#'])

In [ ]:
first =compose(multiply,[proj_prog(1,3), proj_prog(2,3)],3,7) 
second= compose(multiply,[first, proj_prog(3,3)],3,14)

In [ ]:
onesharp(second,['11','#1','11'])

This is undefined.
The register contents at the end are shown below.


,contents
1,11
2,#1
3,11
4,#1##1
5,
6,
7,
8,
9,
10,


In [ ]:
len(parse(compare_prog(4,2)))

81

In [ ]:
def pr(g,h,k): 
  # <g,h>(x-bar,0) = g(x-bar)
  # <g,h>(x-bar,n+1) = h(x-bar,n,<g,h>(x-bar,n))
  # k is the number of xs, so the arity of <g,h> is k+1
  # k+2 is the number of arguments to g
  # k+1 is the number of arguments to the function we are building
  #free is large enough so that neither g nor h use it or any larger register.
  # g and h are assumed to preserve their inputs
  length_of_move = 7
  length_of_clear = 4
  length_of_compare = 81
  length_of_s = len(parse(successor_prog(1,5)))
  length_of_h = len(parse(h))
  n1 = max_register(g)
  n2 = max_register(h)
  free = max([n1,n2])
  print(free)
  big_num = (length_of_h + length_of_clear + length_of_move + length_of_s + 
             length_of_compare)
  print('big_num')
  print(len(ones(big_num)))
  p = (move_prog(k+1,free) +
      ones(k+1) + '##' +
      g +
      compare_prog(free,k+1) +
      ones(free+1) + '#####' + ## back transfer goes here to this cases
      '111###' + ones(length_of_clear + 3) + '###' + '1###' +
      clear_prog(free) + 
      ones(big_num + 2) + '###' +
      h + 
      clear_prog(k+2) + 
      move_prog(k+3,k+2)  + 
      successor_prog(k+1,k+3) +
      compare_prog(free,k+1) +
      ones(big_num + 9) + '####'
      )  
  return(p) 

hh = compose(multiply,[proj_prog(1,3), proj_prog(3,3)],3,7)
gg = '111#'
#onesharp(pr(gg,hh,1),['11','11'])

parse_explain(pr(g,h,k))
#onesharp(pr(g,h,k),['11','11'])

AttributeError: ignored

In [ ]:
hh = compose(multiply,[proj_prog(1,3), proj_prog(3,3)],3,7)
gg = '111#'
free = 10
k = 1
print(len(parse(hh + clear_prog(k+2) + 
      move_prog(k+3,k+2)  + 
      successor_prog(k+1,k+3) +
      compare_prog(free,k+1))))

378


In [ ]:
h = compose(multiply,[proj_prog(1,3), proj_prog(3,3)],3,7)
g = '111#'
k = 1
length_of_move = 7
length_of_clear = 4
length_of_compare = 81
length_of_s = 44
length_of_h = len(parse(h))
n1 = max_register(g)
n2 = max_register(h)
free = max([n1,n2])+1
print('free = ' + str(free))
plan = [['top',move_prog(k+1,free) + ones(k+1) + '##' +g ],
        ['second',compare_prog(free,k+1)],
        ['fourth', 'cases', free+1, 'a', 'main_loop', 'end' ],
        ['a', clear_prog(free)],
        ['goto', 'end'],
        ['main_loop',h +clear_prog(k+2) + move_prog(k+3,k+2)  + successor_prog(k+1,k+3) +
            compare_prog(free,k+1)],
        ['goto', 'fourth'],
       ]
trial = sanity(plan)
#onesharp(trial,['11','#1'])
onesharp(h +clear_prog(k+2) + move_prog(k+3,k+2)  + successor_prog(k+1,k+3) +
            compare_prog(free,k+1) #+
         #h +
         #clear_prog(k+3) + move_prog(k+4,k+3)  + successor_prog(k+1,k+4)# +
            #compare_prog(free,k+1)
            ,['11','1','11','','','', '', '','','', '11'])

free = 11
This is undefined.
The register contents at the end are shown below.


,contents
1,11
2,#1
3,1##1
4,
5,
6,
7,
8,
9,
10,


In [ ]:
len(parse(trial))

478

In [ ]:
onesharp(trial,['#11','11'])

This is undefined.
The register contents at the end are shown below.


,contents
1,#11
2,11
3,###11#11
4,
5,
6,
7,
8,
9,
10,


In [ ]:
onesharp(compare_prog(4,1),['1','', '', '#'])

This is undefined.
The register contents at the end are shown below.


,contents
1,1
2,
3,
4,#
5,1
6,


In [ ]:
h = compose(multiply,[proj_prog(1,3), proj_prog(3,3)],3,7)
h

'1#####11111111###1111###1111##11111##11111####1111#11111#11111111####11111#####111111###111###1##1111####1#111111####1111#####111111###111###11111111##1111####11111111#111111####111#####11111111###1111###1111##11111##11111####1111#11111#11111111####11111#####111111###111###111##1111####111#111111####1111#####111111###111###111111111##1111####111111111#111111####11111111#####111111###111###1111##1111####1111#111111####111111111#####111111###111###11111##1111####11111#111111####111111##1111111##11111#####11111111###1111###11111111##111111111##11111####11111111#111111111#11111111####111111111#####111111###111###11111##1111####11111#111111####111111#####11111111###1111###111111111##1111111111##11111####111111111#1111111111#11111111####1111111111#####111111###111###111111##1111####111111#111111####11111111#####111111###111111111###111111111#####11111111111###1111111111###111111####111111111#####111111111111111###111111###11111###111111111#####111###1111111111111####1###11111111#####111###1

In [ ]:
onesharp(h,['1','11','#1'])

This is undefined.
The register contents at the end are shown below.


,contents
1,1
2,11
3,#1
4,#1
5,
6,
7,
8,
9,
10,


In [ ]:
# <g,h>(x-bar,0) = g(x-bar)
# <g,h>(x-bar,n+1) = h(x-bar,n,<g,h>(x-bar,n))
# k is the number of xs, so the arity of <g,h> is k+1
# k+2 is the number of arguments to g
# k+1 is the number of arguments to the function we are building
#free is large enough so that neither g nor h use it or any larger register.
# g and h are assumed to preserve their inputs
def primitive_recursion(g,h,k):
  length_of_move = 7
  length_of_clear = 4
  length_of_compare = 81
  length_of_s = 44
  length_of_h = len(parse(h))
  n1 = max_register(g)
  n2 = max_register(h)
  free = max([n1,n2])+1
  plan = [['top',move_prog(k+1,free) + g + move_prog(k+1,k+2) + ones(k+1) + '##' ],
          ['second',compare_prog(free,k+1)],
          ['fourth', 'cases', free+1, 'a', 'main_loop', 'end' ],
          ['a', clear_prog(free)],
          ['goto', 'end'],
          ['main_loop',h +clear_prog(k+2) + move_prog(k+3,k+2)  + successor_prog(k+1,k+3) +
            compare_prog(free,k+1)],
          ['goto', 'fourth'],
        ]
  trial = sanity(plan)
  return(trial)

h = compose(multiply,[proj_prog(1,3), proj_prog(3,3)],3,7)
h = compose(s,[proj_prog(3,3)],3,7)
g = '11#'
k = 1
n1 = max_register(g)
n2 = max_register(h)
free = max([n1,n2])+1
#onesharp(move_prog(k+1,free) + g + move_prog(k+1,k+2) + ones(k+1) + '##', ['11', '#1'])
#exp = primitive_recursion(g,h,k)
#onesharp(h,['11','#1','##'])
onesharp(s,['11'])

'##1'

In [ ]:
onesharp(proj_prog(1,1),['11',''])

This is undefined.
The register contents at the end are shown below.


,contents
1,11
2,11
3,


In [ ]:
g = proj_prog(1,1)
h = compose(s,[proj_prog(3,3)],3,8)
k = 1
plus = primitive_recursion(g,h,k)
onesharp(plus,['11','#1'])
#s

This is undefined.
The register contents at the end are shown below.


,contents
1,11
2,#1
3,
4,
5,
6,
7,
8,
9,
10,


In [ ]:
onesharp(exp,['11','#1'])

This is undefined.
The register contents at the end are shown below.


,contents
1,11
2,#1
3,1##1
4,
5,
6,
7,
8,
9,
10,


In [ ]:
def compose(outer_fn,proglist,argument_number,free_reg):
  ## note that the outer_fn has to be 'full', not 'in place'!
  k = len(proglist) # number of functions
  arg = argument_number # = common arity in the proglist
  a = [proglist[i] + move_prog(arg+1,free_reg+i+1) for i in range(0,k)]
  b = unparse(a)
  c = [move_prog(free_reg+i+1,argument_number+i+1) for i in range(0,k)]
  d = unparse(c)
  f = bump(outer_fn,argument_number)
  e = b + d + f   
  return(e)

In [ ]:
#compose(multiply,[proj_prog(1,3), proj_prog(3,3)],3,7)
outer_fn = multiply
proglist =[proj_prog(1,3), proj_prog(3,3)]
#outer_fn = s
#proglist = [proj_prog(3,3)]
argument_number = 3
free_reg = 5
k = len(proglist) # number of functions
arg = argument_number # = common arity in the proglist
a = [proglist[i] + move_prog(arg+1,free_reg+i+1) for i in range(0,k)]
b = unparse(a)
c = [move_prog(free_reg+i+1,argument_number+i+1) for i in range(0,k)]
i = 0
#c= [move_prog(free_reg+i+1,k+i+2)]
d = unparse(c)
f = bump(outer_fn,argument_number)
e = b + d + f  
onesharp(b+d+f,['11','#1','#11'])

This is undefined.
The register contents at the end are shown below.


,contents
1,11
2,#1
3,#11
4,#1##1
5,
6,
7,
8,
9,
10,


In [ ]:
for b in range(0,5):
  print(b)

0
1
2
3
4


In [ ]:
#compose(multiply,[proj_prog(1,3), proj_prog(3,3)],3,7)
#outer_fn = multiply
#proglist =[proj_prog(1,3), proj_prog(3,3)]
outer_fn = s
proglist = [proj_prog(3,3)]
argument_number = 3
free_reg = 5
k = len(proglist) # number of functions
arg = argument_number # = common arity in the proglist
a = [proglist[i] + move_prog(arg+1,free_reg+i+1) for i in range(0,k)]
b = unparse(a)
i = 0
c= [move_prog(free_reg+i+1,argument_number+i+1)]
#c = [move_prog(free_reg+i+1,k+i+2) for i in range(0,k)]
d = unparse(c)
f = bump(outer_fn,argument_number)
e = b + d + f  
onesharp(b+d+f,['11','#1','#11'])

This is undefined.
The register contents at the end are shown below.


,contents
1,11
2,#1
3,#11
4,111
5,
6,


In [ ]:
w = compose(s,[proj_prog(3,3)],3, 6)

In [ ]:
onesharp(w,['11','#1','#11'])

This is undefined.
The register contents at the end are shown below.


,contents
1,11
2,#1
3,#11
4,111
5,
6,
7,


In [ ]:
pre_add = primitive_recursion(proj_prog(1,1),w,1)

In [ ]:
onesharp(pre_add,['11','11'])

This is undefined.
The register contents at the end are shown below.


,contents
1,11
2,11
3,#11
4,
5,
6,
7,
8,
9,
10,


In [ ]:
z_prog = '11##'
#factoid = primitive_recursion(compose(s,[z_prog],1,2), compose(multiply,[proj_prog(3,3),compose(s,[proj_prog(2,2)],1,2)]),2,6)
first = compose(s,[proj_prog(2,3)],3,6)
second =compose(multiply,[proj_prog(3,3), first], 3, 8)
factoid = primitive_recursion(compose(s,[z_prog],1,2),second,2)

In [ ]:
onesharp(factoid,['11','#1'])

KeyboardInterrupt: ignored

In [ ]:
z_prog = '11##'
g = compose(s,[z_prog],1,2)
#factoid = primitive_recursion(compose(s,[z_prog],1,2), compose(multiply,[proj_prog(3,3),compose(s,[proj_prog(2,2)],1,2)]),2,6)
first = compose(s,[proj_prog(2,3)],3,6)
second =compose(multiply,[proj_prog(3,3), first], 3, 8)
h = second
k = 1
length_of_move = 7
length_of_clear = 4
length_of_compare = 81
length_of_s = 44
length_of_h = len(parse(h))
n1 = max_register(g)
n2 = max_register(h)
free = max([n1,n2])+1
plan = [['top',move_prog(k+1,free) + g + move_prog(k+1,k+2) + ones(k+1) + '##' ],
          ['second',compare_prog(free,k+1)],
          ['fourth', 'cases', free+1, 'a', 'main_loop', 'end' ],
          ['a', clear_prog(free)],
          ['goto', 'end'],
          ['main_loop',h +clear_prog(k+2) + move_prog(k+3,k+2)  + successor_prog(k+1,k+3) +
            compare_prog(free,k+1)],
          ['goto', 'fourth'],
        ]
trial = sanity(plan)

fac = primitive_recursion(g,second,1)
onesharp(fac,['11','##1'])
#onesharp(trial,['11','##1'])

This is undefined.
The register contents at the end are shown below.


,contents
1,11
2,##1
3,###11
4,
5,
6,
7,
8,
9,
10,


In [ ]:
first = compose(s,[proj_prog(1,2)],2,6)
second =compose(multiply,[proj_prog(2,2), first], 2, 8)
fac = primitive_recursion('1#',second,0)
onesharp(fac,['##1'])

This is undefined.
The register contents at the end are shown below.


,contents
1,##1
2,###11
3,
4,
5,
6,
7,
8,
9,
10,


In [ ]:
onesharp(trial, ['#1','#11'])

This is undefined.
The register contents at the end are shown below.


,contents
1,#1
2,#11
3,####1#11#1
4,
5,
6,
7,
8,
9,
10,


In [ ]:
1011010000